In [ ]:
df_full = pd.read_csv(output_process+'An Giang'+'_train_preprocessed.csv', parse_dates=True, index_col= None, encoding = 'unicode_escape')

scaler = joblib.load(output_process+'An Giang'+'_train_scalerMinMaxNorm.save') #ok
# trainlist = [x for x in df_full.Dengue_fever_rates]
nstep = args.n_predicted_period_months

df_use =pd.DataFrame()
df_use[f'{feature_list[0]}_lag1'] = df_full[feature_list[0]].shift(1)
df_use[f'{feature_list[0]}_lag2'] = df_full[feature_list[0]].shift(2)
df_use[f'{feature_list[0]}_lag3'] = df_full[feature_list[0]].shift(3)
df_use[f'{feature_list[1]}_lag1'] = df_full[feature_list[1]].shift(1)
df_use[f'{feature_list[1]}_lag2'] = df_full[feature_list[1]].shift(2)
df_use[f'{feature_list[1]}_lag3'] = df_full[feature_list[1]].shift(3)
df_use[f'{feature_list[2]}_lag1'] = df_full[feature_list[2]].shift(1)
df_use[f'{feature_list[2]}_lag2'] = df_full[feature_list[2]].shift(2)
df_use[f'{feature_list[2]}_lag3'] = df_full[feature_list[2]].shift(3)
df_use.dropna(inplace=True)
trainlist = [x for x in df_full[args.look_back:].Dengue_fever_rates]

X = df_use[[f'{feature_list[0]}_lag1',f'{feature_list[0]}_lag2',f'{feature_list[0]}_lag3',
                f'{feature_list[1]}_lag1',f'{feature_list[1]}_lag2',f'{feature_list[1]}_lag3',
                f'{feature_list[2]}_lag1',f'{feature_list[2]}_lag2',f'{feature_list[2]}_lag3']]
exog_list = X.values.tolist()
print(len(exog_list))
print(len(trainlist))
predict_list = []
for i in range (args.test_size+(nstep-1)):
    history = trainlist[:-args.test_size-(nstep-1-i)]
    related_exog = exog_list[:-args.test_size-(nstep-1-i)]
    print("")
    print(len(history))
    print(len(related_exog))
    # print(related_exog.shape)
    model = SARIMAX(history, 
                    exog=related_exog,
                    order=order, 
                    seasonal_order=sorder, 
                    trend = trend,
                    enforce_stationarity=False,
                    enforce_invertibility=False)
    model_fit = model.fit(disp=False)
    nstep = 6
    yhat = model_fit.predict(len(history), len(history) + nstep - 1, exog=related_exog)
    yhat = model_fit.predict(len(history), len(history) + nstep - 1)

    predict_list.append(yhat)

y_pred_list = []
for step in range(nstep):
    moving = nstep-1-step
    y_pred_list.append([x[step] for x in predict_list][moving:args.test_size+moving])

df_eval_true_inverse = df_full[-args.test_size:]
y_true = scaler.inverse_transform(df_eval_true_inverse.iloc[:,:-1])[:,[-1]].reshape(args.test_size)

y_pred_inverse_list = []
for step in range(nstep):
    df_eval_pred_inverse = df_full[-args.test_size:]
    df_eval_pred_inverse[args.labels]= y_pred_list[step] #step 1
    y_pred_inverse = scaler.inverse_transform(df_eval_pred_inverse.iloc[:,:-1])[:,[-1]].reshape(args.test_size)
    y_pred_inverse_list.append(y_pred_inverse)

y_pred_inverse_list
df_compare_test_predict = pd.DataFrame({
    'y_true':y_true,
    'y_pred_1step':y_pred_inverse_list[0],
    'y_pred_2step':y_pred_inverse_list[1],
    'y_pred_3step':y_pred_inverse_list[2],
    'y_pred_4step':y_pred_inverse_list[3],
    'y_pred_5step':y_pred_inverse_list[4],
    'y_pred_6step':y_pred_inverse_list[5],
    })
df_compare_test_predict.plot()
plt.legend()
plt.show()